# Step 1: Save frames of the video

In [2]:
import os
video_path = "Peter.mp4"
root_dir = "./my_dataset/Peter_chess"
img_dir = "Camera_B1"

img_size = 1024 #zju_mocap is 1024



In [ ]:
import cv2

# Open the video file
video = cv2.VideoCapture(video_path)
# Check if the video file was successfully opened
if not video.isOpened():
    print('Error opening video file')
# Create a directory to save the extracted frames
output_directory = os.path.join(root_dir, "Camera_B1bak")
os.makedirs(output_directory, exist_ok=True)

# Read and save frames from the video
frame_count = 0
while True:
    # Read the next frame from the video
    ret, frame = video.read()

    # Break the loop if no frame is read
    if not ret:
        break

    # Save the frame as an image
    frame_path = os.path.join(output_directory, f'frame_{frame_count}.png')
    cv2.imwrite(frame_path, frame)

    # Increment the frame count
    frame_count += 1

# Release the video file
video.release()

# delete some bad images manually and then rename the images according to their sort

In [29]:

import os
import re
import glob

def reorder_dir(directory, surfix = ".png"):
    # Get a list of all files in the directory
    files = os.listdir(directory)

    # Filter only image files (you can add more extensions if needed)
    image_files = [file for file in files if re.match(r'.*\.(jpg|png)$', file, re.I)]

    # Sort the image files based on the main name before the suffix
    sorted_files = sorted(image_files, key=lambda x: re.split(r'(\d+)', os.path.splitext(x)[0]))

    # Rename the sorted files
    for i, file in enumerate(sorted_files):
        # surfix = os.path.splitext(file)[1]
        new_name = f'{i:05d}{surfix}'
        os.rename(os.path.join(directory, file), os.path.join(directory, new_name))

In [ ]:
#a image's height is larger. pad the the images to be square and then resize to 1024*1024

In [5]:
from PIL import Image
import os


directory = os.path.join(root_dir, "Camera_B1bak")
# Get a list of all files in the directory
files = os.listdir(directory)

# Filter only image files (you can add more extensions if needed)
image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

# Process each image
for file in image_files:
    # Open the image
    image_path = os.path.join(directory, file)
    image = Image.open(image_path)

    # Calculate the desired size
    max_size = max(image.size)
    new_size = (max_size, max_size)

    # Create a new blank square image
    padded_image = Image.new('RGB', new_size)
    # Paste the original image onto the blank image, centered
    padded_image.paste(image, ((max_size - image.size[0]) // 2, (max_size - image.size[1]) // 2))

    # Resize the padded image to 1024x1024
    resized_image = padded_image.resize((img_size, img_size))

    # Save the resized image with a new name
    new_file = file.split(".")[0] + ".png"
    new_path = os.path.join(directory, new_file)
    resized_image.save(new_path)

KeyboardInterrupt: 

# Step 2: Segment the images to get mask using segment everything
We fintune the segment model on single predefined mask to make it more accurately segment the person from the background.
We use photoshop to get the predefined mask and put it under the Annotations folder. The reference image name is {ref_idx:5d}.png. 
See readme of Personalize-SAM

In [3]:
os.makedirs(os.path.join(root_dir, "masks"), exist_ok=True)
os.makedirs(os.path.join(root_dir, "masks", img_dir), exist_ok=True)
os.makedirs(os.path.join(root_dir, "Annotations"), exist_ok=True)


In [4]:

mask_dir = os.path.join("masks", img_dir)
!python Personalize-SAM/persam_f.py --rootdir {root_dir} --refdir "Annotations" --imgdir {"Camera_B1"} --outdir {mask_dir} --ref_idx 13 --ckpt './Personalize-SAM/weights/sam_vit_h_4b8939.pth' 

^C
Traceback (most recent call last):
  File "/home/yjli/AIGC/Adversarial_camou/Personalize-SAM/persam_f.py", line 10, in <module>
    import matplotlib.pyplot as plt
  File "/home/yjli/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/matplotlib/pyplot.py", line 66, in <module>
    from matplotlib.figure import Figure, FigureBase, figaspect
  File "/home/yjli/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/matplotlib/figure.py", line 43, in <module>
    from matplotlib import _blocking_input, backend_bases, _docstring, projections
  File "/home/yjli/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/matplotlib/projections/__init__.py", line 55, in <module>
    from .. import axes, _docstring
  File "/home/yjli/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/matplotlib/axes/__init__.py", line 2, in <module>
    from ._axes import *
  File "/home/yjli/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/matplotlib/axes/_axes.py", line 31, in <module>
    import matplotlib

# using SPIN to get croped images , the SMPL parameters, bbox and camera parameters

In [9]:
# crop the mask images using bbox
import json
import numpy as np
import torch
import numpy as np
import scipy.misc
import cv2

def get_transform(center, scale, res, rot=0):
    """Generate transformation matrix."""
    h = 200 * scale
    t = np.zeros((3, 3))
    t[0, 0] = float(res[1]) / h
    t[1, 1] = float(res[0]) / h
    t[0, 2] = res[1] * (-float(center[0]) / h + .5)
    t[1, 2] = res[0] * (-float(center[1]) / h + .5)
    t[2, 2] = 1
    if not rot == 0:
        rot = -rot # To match direction of rotation from cropping
        rot_mat = np.zeros((3,3))
        rot_rad = rot * np.pi / 180
        sn,cs = np.sin(rot_rad), np.cos(rot_rad)
        rot_mat[0,:2] = [cs, -sn]
        rot_mat[1,:2] = [sn, cs]
        rot_mat[2,2] = 1
        # Need to rotate around center
        t_mat = np.eye(3)
        t_mat[0,2] = -res[1]/2
        t_mat[1,2] = -res[0]/2
        t_inv = t_mat.copy()
        t_inv[:2,2] *= -1
        t = np.dot(t_inv,np.dot(rot_mat,np.dot(t_mat,t)))
    return t

def transform(pt, center, scale, res, invert=0, rot=0):
    """Transform pixel location to different reference."""
    t = get_transform(center, scale, res, rot=rot)
    if invert:
        t = np.linalg.inv(t)
    new_pt = np.array([pt[0]-1, pt[1]-1, 1.]).T
    new_pt = np.dot(t, new_pt)
    return new_pt[:2].astype(int)+1

def scale_box(bbox):
    """Get center and scale of bounding box from bounding box annotations.
    The expected format is [top_left(x), top_left(y), width, height].
    """
    ul_corner = bbox[:2]
    center = [ul_corner[0] + 0.5 * bbox[2], ul_corner[1] + 0.5 * bbox[3]]
    width = max(bbox[2], bbox[3])
    scale = width / 200.0
    # make sure the bounding box is rectangular
    return center, scale

def crop(img, center, scale, res, rot=0):
    """Crop image according to the supplied bounding box."""
    # Upper left point
    ul = np.array(transform([1, 1], center, scale, res, invert=1))-1
    # Bottom right point
    br = np.array(transform([res[0]+1, 
                             res[1]+1], center, scale, res, invert=1))-1
    
    # Padding so that when rotated proper amount of context is included
    pad = int(np.linalg.norm(br - ul) / 2 - float(br[1] - ul[1]) / 2)
    if not rot == 0:
        ul -= pad
        br += pad

    new_shape = [br[1] - ul[1], br[0] - ul[0]]
    if len(img.shape) > 2:
        new_shape += [img.shape[2]]
    new_img = np.zeros(new_shape)

    # Range to fill new array
    new_x = max(0, -ul[0]), min(br[0], len(img[0])) - ul[0]
    new_y = max(0, -ul[1]), min(br[1], len(img)) - ul[1]
    # Range to sample from original image
    old_x = max(0, ul[0]), min(len(img[0]), br[0])
    old_y = max(0, ul[1]), min(len(img), br[1])
    new_img[new_y[0]:new_y[1], new_x[0]:new_x[1]] = img[old_y[0]:old_y[1], 
                                                        old_x[0]:old_x[1]]

    if not rot == 0:
        # Remove padding
        new_img = scipy.misc.imrotate(new_img, rot)
        new_img = new_img[pad:-pad, pad:-pad]

    new_img = cv2.resize(new_img, res)
    return new_img

inputfolder = "./my_dataset/Peter_chess/masks_bak/Camera_B1"
outputfolder = "./my_dataset/Peter_chess/masks/Camera_B1"

with open(os.path.join(root_dir, 'metadata.json'), 'r') as f:
    frame_infos = json.load(f)
    
    for frame_name in frame_infos:
        try:
            img = cv2.imread(os.path.join(inputfolder, frame_name))
            cam_body_info = frame_infos[frame_name]
            bbox = cam_body_info['bbox']
            center, scale = scale_box(bbox)
            img = crop(img, center, scale, (1024, 1024))
            cv2.imwrite(os.path.join(outputfolder, frame_name), img)
        except:
            os.remove(f"{root_dir}/" + "Camera_B1crop" + "/" + frame_name) 


00000.png
(1024, 1024, 3)
00001.png
(1024, 1024, 3)
00002.png
(1024, 1024, 3)
00003.png
00004.png
(1024, 1024, 3)
00005.png
(1024, 1024, 3)
00006.png
(1024, 1024, 3)
00007.png
(1024, 1024, 3)
00008.png
(1024, 1024, 3)


[ WARN:0@550.391] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00003.png'): can't open/read file: check file path/integrity


00009.png
(1024, 1024, 3)
00010.png
(1024, 1024, 3)
00011.png
(1024, 1024, 3)
00012.png
(1024, 1024, 3)
00013.png
(1024, 1024, 3)
00014.png
(1024, 1024, 3)
00015.png
(1024, 1024, 3)
00016.png
(1024, 1024, 3)
00017.png
(1024, 1024, 3)
00018.png
(1024, 1024, 3)
00019.png
(1024, 1024, 3)
00020.png
(1024, 1024, 3)
00021.png
(1024, 1024, 3)
00022.png
(1024, 1024, 3)
00023.png
(1024, 1024, 3)
00024.png
(1024, 1024, 3)
00025.png
(1024, 1024, 3)
00026.png
(1024, 1024, 3)
00027.png
(1024, 1024, 3)
00028.png
(1024, 1024, 3)
00029.png
(1024, 1024, 3)
00030.png
(1024, 1024, 3)
00031.png
(1024, 1024, 3)
00032.png
(1024, 1024, 3)
00033.png
(1024, 1024, 3)
00034.png
(1024, 1024, 3)
00035.png
(1024, 1024, 3)
00036.png
(1024, 1024, 3)
00037.png
(1024, 1024, 3)
00038.png
(1024, 1024, 3)
00039.png
(1024, 1024, 3)
00040.png
(1024, 1024, 3)
00041.png
(1024, 1024, 3)
00042.png
(1024, 1024, 3)
00043.png
(1024, 1024, 3)
00044.png
(1024, 1024, 3)
00045.png
(1024, 1024, 3)
00046.png
(1024, 1024, 3)
00047.png
(1

[ WARN:0@561.080] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00427.png'): can't open/read file: check file path/integrity
[ WARN:0@561.212] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00432.png'): can't open/read file: check file path/integrity


(1024, 1024, 3)
00435.png
00436.png
00437.png
00438.png
00439.png
00440.png
00441.png
(1024, 1024, 3)
00442.png
(1024, 1024, 3)
00443.png
(1024, 1024, 3)
00444.png
(1024, 1024, 3)
00445.png
(1024, 1024, 3)
00446.png


[ WARN:0@561.285] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00435.png'): can't open/read file: check file path/integrity
[ WARN:0@561.287] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00436.png'): can't open/read file: check file path/integrity
[ WARN:0@561.287] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00437.png'): can't open/read file: check file path/integrity
[ WARN:0@561.288] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00438.png'): can't open/read file: check file path/integrity
[ WARN:0@561.288] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00439.png'): can't open/read file: check file path/integrity
[ WARN:0@561.288] global loadsave.cpp:248 findDecoder imread_('./my_dataset/Peter_chess/masks_bak/Camera_B1/00440.png'): can't open/read file: check file p

(1024, 1024, 3)
00447.png
(1024, 1024, 3)
00448.png
(1024, 1024, 3)
00449.png
(1024, 1024, 3)
00450.png
(1024, 1024, 3)
00451.png
(1024, 1024, 3)
00452.png
(1024, 1024, 3)
00453.png
(1024, 1024, 3)
00454.png
(1024, 1024, 3)
00455.png
(1024, 1024, 3)
00456.png
(1024, 1024, 3)
00457.png
(1024, 1024, 3)
00458.png
(1024, 1024, 3)
00459.png
(1024, 1024, 3)
00460.png
(1024, 1024, 3)
00461.png
(1024, 1024, 3)
00462.png
(1024, 1024, 3)
00463.png
(1024, 1024, 3)
00464.png
(1024, 1024, 3)
00465.png
(1024, 1024, 3)
00466.png
(1024, 1024, 3)
00467.png
(1024, 1024, 3)
00468.png
(1024, 1024, 3)
00469.png
(1024, 1024, 3)
00470.png
(1024, 1024, 3)
00471.png
(1024, 1024, 3)
00472.png
(1024, 1024, 3)
00473.png
(1024, 1024, 3)
00474.png
(1024, 1024, 3)
00475.png
(1024, 1024, 3)
00476.png
(1024, 1024, 3)
00477.png
(1024, 1024, 3)
00478.png
(1024, 1024, 3)
00479.png
(1024, 1024, 3)
00480.png
(1024, 1024, 3)
00481.png
(1024, 1024, 3)
00482.png
(1024, 1024, 3)
00483.png
(1024, 1024, 3)
00484.png
(1024, 1024,

# each run will shrink the mask a little, so please first run Personalize-SAM

In [10]:
import cv2
import numpy as np
from PIL import Image  

# os.rename(os.path.join(root_dir, img_dir), os.path.join(root_dir, img_dir+"bak"))

masks_path = os.path.join(root_dir, "masks", img_dir)
file_list = sorted(os.listdir(masks_path))

frame_path = os.path.join(root_dir, img_dir+"crop")
new_frame_path = os.path.join(root_dir, img_dir)
# !rm -rf {new_frame_path}
os.makedirs(new_frame_path, exist_ok=True)

for image_name in file_list:
    
    mask_path =os.path.join(masks_path,image_name)
    mask = cv2.imread(mask_path)
    kernel = np.ones((3, 3), dtype=np.uint8)
    erosion = cv2.erode(mask, kernel, iterations=1) # # remove the white edge
    cv2.imwrite(os.path.join(mask_path, image_name), erosion)
    
    frame = cv2.imread(os.path.join(frame_path, image_name))
    new_frame = (np.array(frame) * (np.array(erosion)/255))
    cv2.imwrite(os.path.join(new_frame_path, image_name), new_frame)

# prepare the IUV images

In [11]:

import os

os.system(f"python  /home/yjli/AIGC/Adversarial_camou/detectron2/projects/DensePose/apply_net.py dump \
    /home/yjli/AIGC/Adversarial_camou/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml \
    https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl \
    {os.path.join(root_dir, img_dir)} --output {root_dir}/{img_dir}_dump.pkl -v")


[04/28 05:58:11 apply_net]: Loading config from /home/yjli/AIGC/Adversarial_camou/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml
[04/28 05:58:11 apply_net]: Loading model from https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl
[04/28 05:58:14 apply_net]: Loading data from ./my_dataset/Peter_chess/Camera_B1


/home/yjli/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/28 05:58:15 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00206.png
[04/28 05:58:15 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00044.png
[04/28 05:58:15 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00058.png
[04/28 05:58:15 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00047.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00461.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00453.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00350.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00119.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00454.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00352.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00456.png
[04/28 05:58:16 apply_net]: Processing ./my_dataset/Peter_chess/Camera_B1/00

0

In [12]:


# extract the IUV information
import sys
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import functional as TF
import cv2
import shutil
sys.path.append(f"/home/yjli/AIGC/Adversarial_camou/detectron2/projects/DensePose/")

def get_iuvimg(root_dir, img_dir):
    # for index in range(1, 24):
    f = open(f"{root_dir}/{img_dir}_dump.pkl", 'rb')
    data = torch.load(f)
    if os.path.exists(f"{root_dir}/densepose/" + img_dir):
        shutil.rmtree(f"{root_dir}/densepose/" + img_dir)
    os.makedirs(f"{root_dir}/densepose/" + img_dir)

    for entry in data:
        file_path = entry['file_name']
        filename = file_path.split('/')[-1]  # Splitting the path by '/' and getting the last element

        # Example: revising the path
        revised_path = f"{root_dir}/densepose/" + img_dir + "/" + filename.split(".")[0] +"_IUV.png"
        # print(revised_path)
        
        try:
            labels = entry['pred_densepose'][0].labels #[1, 24] , 568*268
            uv = entry['pred_densepose'][0].uv #[0,1], [2, 568, 268]
            # Reshape the labels tensor to match the uv tensor shape
            labels = labels.unsqueeze(0)  # Add a new dimension at the beginning

            # Concatenate the labels and uv tensors along the channel dimension
            iuv_images = torch.cat([labels, uv*255], dim=0)

            pred_boxes_XYXY = entry['pred_boxes_XYXY'][0]
            # Convert the bounding box coordinates to integers
            pred_boxes_XYXY = pred_boxes_XYXY.int()
            
            padding_left = pred_boxes_XYXY[0]
            padding_top = pred_boxes_XYXY[1]
            padding_right = img_size - pred_boxes_XYXY[2]
            padding_bottom = img_size - pred_boxes_XYXY[3]
            iuv_images = TF.pad( iuv_images , (padding_left, padding_top, padding_right, padding_bottom))
            iuv_images = TF.resize(iuv_images, (img_size, img_size))

            # Convert the tensor to NumPy array
            iuv_np = iuv_images.cpu().numpy()
            iuv_np = np.transpose(iuv_np, (1, 2, 0))

            cv2.imwrite(revised_path, iuv_np)
        except: 
            if os.path.exists(f"{root_dir}/" + img_dir + "/" + filename):
                os.remove(f"{root_dir}/" + img_dir + "/" + filename) 
            if os.path.exists(f"{root_dir}/masks/" + img_dir + "/" + filename):
                os.remove(f"{root_dir}/masks/" + img_dir + "/" + filename)
                
!rm -rf {os.path.join( root_dir,  "densepose")}               
outpath =os.path.join( root_dir,  "densepose")
os.makedirs(outpath , exist_ok=True)
get_iuvimg(root_dir, img_dir)
# 获取文件夹中的所有文件
files = os.listdir(f"{root_dir}/masks/" + img_dir)
# 统计文件个数
file_count1 = len(files)
# 获取文件夹中的所有文件
files = os.listdir(f"{root_dir}/" + img_dir)
# 统计文件个数
file_count2 = len(files)
print(file_count1, file_count2)
           


rm: cannot remove 'rf': No such file or directory
rm: cannot remove './my_dataset/Peter_chess/densepose': Is a directory
676 676


# get the smpl params (before that, you should get the vertices and pred smple rotmat through SPIN)


In [13]:
# get the smpl params (before that, you should get the vertices and pred smple rotmat through SPIN)

import os
import sys

import json
import yaml
import pickle
import numpy as np
from tqdm import tqdm

from pathlib import Path
sys.path.append(str(Path(os.getcwd()).resolve().parents[1]))
sys.path.append("/home/yjli/AIGC/humannerf/")
from third_parties.smpl.smpl_numpy import SMPL
MODEL_DIR = '/home/yjli/AIGC/humannerf/third_parties/smpl/models'

# !rm -rf {os.path.join(root_dir, "params")}

def inv_rodrigues(Rotation):
    '''
    input: Rotation Matrix of (None, 3, 3)
    output: Joint Locations (None, 3)
    '''
    inv_r = []
    trace_r = []
    angle_w = []
    for i in range(Rotation.shape[0]):
        invr = Rotation[i][2, 1] - Rotation[i][1, 2], Rotation[i][0,
                                                                  2] - Rotation[i][2, 0], Rotation[i][1, 0] - Rotation[i][0, 1]
        tacer = Rotation[i][0, 0] + Rotation[i][1, 1] + Rotation[i][2, 2]
        s = np.sqrt(tacer + 1)
        inv_r.append(invr)
        trace_r.append(tacer)
        if tacer >= 3-np.finfo(np.float64).eps:
            angelw = (0.5 - ((tacer-3) / 12)) * np.array(invr)
        elif tacer < 3 - np.finfo(np.float64).eps and tacer > -1 + np.finfo(np.float64).eps:
            theta_w = np.arccos((tacer - 1) / 2)
            angelw = (theta_w / (2 * np.sin(theta_w))) * np.array(invr)
        elif tacer <= -1 + np.finfo(np.float64).eps:
            v_a = s/2
            v_b = (1/2*s) * (Rotation[i][1, 0] + Rotation[i][0][1])
            v_c = (1/2*s) * (Rotation[i][2, 0] + Rotation[i][0, 2])
            angelw = np.pi * (np.array((v_a, v_b, v_c)) *
                              np.linalg.norm(np.array((v_a, v_b, v_c))))
        angle_w.append(angelw)
    inv_r = np.array(inv_r)
    trace_r = np.array(trace_r)
    angle_w = np.array(angle_w)
    return angle_w


def main():

    sex = "neutral"
    subject_dir = root_dir
    output_path = subject_dir
    
    os.makedirs(os.path.join(root_dir, "params"), exist_ok=True)
    
    frame_infos = []
    with open(os.path.join(subject_dir, 'metadata.json'), 'r') as f:
        frame_infos = json.load(f)

    smpl_model = SMPL(sex=sex, model_dir=MODEL_DIR)

    cameras = {}
    mesh_infos = {}
    all_betas = []
    for frame_name in tqdm(frame_infos):
        cam_body_info = frame_infos[frame_name]
        poses_rotmat = np.array(cam_body_info['body_pose'], dtype=np.float32) #[1, 23, 3, 3]
        global_orient =  np.array(cam_body_info['global_orient'], dtype=np.float32) #[1, 1, 3, 3]
        
        poses_rotmat = np.concatenate((global_orient, poses_rotmat), axis = 1)[0] #[24, 3,3]
        
        poses = inv_rodrigues(poses_rotmat) 
        poses = np.squeeze(poses.reshape(-1,1))
        
        betas = np.array(cam_body_info['betas'], dtype=np.float32)[0] #(10, )
        K = np.array(cam_body_info['K'], dtype=np.float32)
        D = np.array(cam_body_info['D'], dtype=np.float32)
        R = np.array(cam_body_info['R'], dtype=np.float32)
        T = np.array(cam_body_info['T'], dtype=np.float32)
        # D = np.array(cam_body_info['distortion'], dtype=np.float32)
        
        all_betas.append(betas)

        ##############################################
        # Below we tranfer the global body rotation to camera pose

        # Get T-pose joints
        _, tpose_joints = smpl_model(np.zeros_like(poses), betas)

        # get global Rh, Th
        pelvis_pos = tpose_joints[0].copy()
        Th = pelvis_pos
        Rh = poses[:3].copy()
        # Rh = global_orient[0][0].copy() # [3, 3]

        # get refined T-pose joints
        tpose_joints = tpose_joints - pelvis_pos[None, :]

        # remove global rotation from body pose
        poses[:3] = 0
        # poses[0,:,:] = np.eye(3)

        # get posed joints using body poses without global rotation
        _, joints = smpl_model(poses, betas)
        joints = joints - pelvis_pos[None, :]

        mesh_infos[frame_name] = {
            'Rh': Rh,
            'Th': Th,
            'poses': [poses],
            'joints': joints,
            'tpose_joints': tpose_joints
        }

        cameras[frame_name] = {
            'K': K,
            'D': D,
            'T': T,
            'R': R
        }
        
        params_name = os.path.join(root_dir, "params", str(int(frame_name.split(".")[0])) +".npy")
        np.save(params_name, mesh_infos[frame_name])

    # write camera infos
    # with open(os.path.join(output_path, 'cameras.pkl'), 'wb') as f:   
    #     pickle.dump(cameras, f)
        
    # write mesh infos
    # with open(os.path.join(output_path, 'mesh_infos.pkl'), 'wb') as f:   
    #     pickle.dump(mesh_infos, f)
        


    # write canonical joints
    """    
    avg_betas = np.mean(np.stack(all_betas, axis=0), axis=0)
    smpl_model = SMPL(sex, model_dir=MODEL_DIR)
    _, template_joints = smpl_model(np.zeros(72), avg_betas)
    with open(os.path.join(output_path, 'canonical_joints.pkl'), 'wb') as f:   
        pickle.dump(
            {
                'joints': template_joints,
            }, f)
    """


if __name__ == '__main__':
    main()

 99%|█████████▉| 679/685 [00:06<00:00, 83.01it/s] /tmp/ipykernel_61662/110035631.py:32: RuntimeWarning: invalid value encountered in sqrt
  s = np.sqrt(tacer + 1)
100%|██████████| 685/685 [00:06<00:00, 107.30it/s]


In [39]:
# 删除只存在于其中一些文件夹中的图片

import os

# 定义文件夹列表
folder_list = ['folder1', 'folder2', 'folder3']

# 创建文件名集合
file_set = set()

# 遍历文件夹列表
for folder in folder_list:
    # 遍历文件夹中的文件
    for filename in os.listdir(folder):
        # 获取文件的完整路径
        file_path = os.path.join(folder, filename)
        # 如果文件是普通文件而不是文件夹
        if os.path.isfile(file_path):
            # 将文件名添加到集合中
            file_set.add(filename)

# 遍历文件夹列表
for folder in folder_list:
    # 遍历文件夹中的文件
    for filename in os.listdir(folder):
        # 获取文件的完整路径
        file_path = os.path.join(folder, filename)
        # 如果文件是普通文件而不是文件夹，并且文件名不在文件名集合中
        if os.path.isfile(file_path) and filename not in file_set:
            # 删除文件
            os.remove(file_path)

# import os
# reorder_dir(f"{root_dir}/densepose/" + img_dir , surfix = "_IUV.png")
# reorder_dir(f"{root_dir}/masks/" + img_dir )
# reorder_dir(f"{root_dir}/" + img_dir )


In [ ]:

# 获取文件夹中的所有文件
files = os.listdir(f"{root_dir}/densepose/" + img_dir)
# 统计文件个数
file_count1 = len(files)
# 获取文件夹中的所有文件
files = os.listdir(f"{root_dir}/" + img_dir)
# 统计文件个数
file_count2 = len(files)
if file_count1 != file_count2:
    outpath =os.path.join( root_dir,  "densepose")
    os.makedirs(outpath , exist_ok=True)
    os.system(f"python  /home/yjli/AIGC/Adversarial_camou/detectron2/projects/DensePose/apply_net.py dump \
        /home/yjli/AIGC/Adversarial_camou/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml \
        https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl \
        {os.path.join(root_dir, img_dir)} --output {root_dir}/{img_dir}_dump.pkl -v")
    get_iuvimg(root_dir, img_dir)
    # 获取文件夹中的所有文件
    files = os.listdir(f"{root_dir}/densepose/" + img_dir)
    # 统计文件个数
    file_count1 = len(files)
    # 获取文件夹中的所有文件
    files = os.listdir(f"{root_dir}/" + img_dir)
    # 统计文件个数
    file_count2 = len(files)
    print(file_count1, file_count2)
    if file_count1==file_count2:
        print("success!")

else:
    print("success!")
    



In [8]:

import os

def count_files(folder_path):
    count = 0
    for _, _, files in os.walk(folder_path):
        count += len(files)
    return count

# 指定文件夹路径
folder_path = '/home/yjli/AIGC/Adversarial_camou/my_dataset/Peter_chess/vertices/Camera_B1'

# 获取文件夹下所有文件的个数
file_count = count_files(folder_path)
print("vertice count", file_count)
# 指定文件夹路径
folder_path = '/home/yjli/AIGC/Adversarial_camou/my_dataset/Peter_chess/Camera_B1'

# 获取文件夹下所有文件的个数
file_count = count_files(folder_path)
print("image count",file_count)


vertice count 685
image count 653


100%|█████████▉| 683/685 [00:07<00:00, 87.22it/s] /tmp/ipykernel_17290/2988346868.py:28: RuntimeWarning: invalid value encountered in sqrt
  s = np.sqrt(tacer + 1)
100%|██████████| 685/685 [00:07<00:00, 95.16it/s]


# Step 3: crop the image and remove the background (not used)

In [10]:
from util.personcroper import PersonCropper
from PIL import Image
import numpy as np
import torch
from torchvision import utils
from tqdm import tqdm
import cv2
cropper = PersonCropper("/home/yjli/AIGC/Flow-Style-VTON/adversarialyolo/weights/yolo.weights", 
                                    "/home/yjli/AIGC/Flow-Style-VTON/adversarialyolo/cfg/yolo.cfg",
                                    "/home/yjli/AIGC/Flow-Style-VTON/adversarialyolo/data/coco.names")
masks_path = os.path.join(root_dir, "mask")
file_list = os.listdir(frames_path)
output_path = os.path.join(root_dir, "cropped")
os.makedirs(output_path, exist_ok=True)

padded_path = os.path.join(root_dir, "padded")
pad_size = [256, 192]
os.makedirs(padded_path, exist_ok=True)

for image_name in tqdm(file_list):
    # load human images from video frame
    frame_path = os.path.join(frames_path, image_name)
    mask_path =os.path.join(masks_path,image_name)
    pil_image = Image.open(frame_path).convert('RGB') #[544, 960]
    mask = cv2.imread(mask_path)
    kernel = np.ones((5, 5), dtype=np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1) # erotion the mask
    mask_array = np.array(mask)/255
    pil_img_array = np.array(pil_image)/255
    pil_img_array = pil_img_array * mask_array + np.ones_like(pil_img_array) * (1-mask_array)

    image = torch.from_numpy(pil_img_array).permute(2,0,1).unsqueeze(0).cuda().float()
    cropped_images = cropper.crop_persons(image)
   
    if len(cropped_images) <= 0:
        continue 
    else:
        h = cropped_images[0].shape[2]
        w = cropped_images[0].shape[3]
        
        padw = int(0.5*(pad_size[1]*h/pad_size[0]-w))
        dim=(padw,padw,0,0)# (左边填充数， 右边填充数， 上边填充数， 下边填充数)
        padded_image = torch.nn.functional.pad(cropped_images[0],dim,"constant",value=255)
        padded_image = torch.nn.functional.interpolate(padded_image, size=pad_size, mode='bilinear', align_corners=False)

        utils.save_image(
            cropped_images[0],
            os.path.join(output_path, image_name),
            nrow=int(1),
            normalize=True,
            value_range=(0,1),
        )
        
        utils.save_image(
            padded_image,
            os.path.join(padded_path, image_name),
            nrow=int(1),
            normalize=True,
            value_range=(0,1),
        )

  1%|          | 2/243 [00:00<00:18, 12.91it/s]

torch.Size([1, 3, 436, 138])
torch.Size([1, 3, 613, 211])
torch.Size([1, 3, 682, 121])


  2%|▏         | 6/243 [00:00<00:18, 12.92it/s]

torch.Size([1, 3, 549, 136])
torch.Size([1, 3, 649, 179])
torch.Size([1, 3, 628, 120])


  3%|▎         | 8/243 [00:00<00:18, 12.99it/s]

...

 99%|█████████▉| 240/243 [00:18<00:00, 13.09it/s]

torch.Size([1, 3, 441, 136])
torch.Size([1, 3, 493, 146])
torch.Size([1, 3, 638, 156])


100%|█████████▉| 242/243 [00:18<00:00, 12.32it/s]

torch.Size([1, 3, 517, 117])
torch.Size([1, 3, 592, 136])
torch.Size([1, 3, 550, 125])


100%|██████████| 243/243 [00:18<00:00, 13.02it/s]
